# Calculate 13C chemical shifts from DCodes

Step 1: Generate SMILES Code for your Molecule

Step 2: Enter the SMILES and generate the 13C chemical shifts.

Step 3: Get your results as formatted output.

Attention: A chemical shift of -999 stands for no results. A  Minimum number of neighbours of 5 is necessary for completing a calculation.

You can use the JSME Editor for generating SMILES Codes - for example https://jsme-editor.github.io/dist/JSME_test.html


In [2]:
######################################################################
# In the first code cell, the data is read into the pandas DataFrame.
# Then it doesn't need to be done again every time.

import pandas as pd
from rdkit import *
from rdkit.Chem import AllChem, Draw
from IPython.display import display, Image, HTML
import os, sys
import tempfile
from dcode import geometry, tools
from dcode.geometry import *
from dcode.tools import *
from dcode.calcshift import *


codefile="codes/v3_update_10_06_2025.csv"

# reading data for calculation
df = pd.read_csv(codefile)



In [3]:
####################################################################################
#                                                                                  #
#              Enter SMILES CODE here                                              #
#                                                                                  #
####################################################################################

smiles='N[C@@H](CCC(=O)N[C@@H](CS)C(=O)NCC(=O)O)C(=O)O'


#########################################
#                                       #
# Do not edit more things !!!!          #
#                                       #
#########################################



groesse=(450,450)   # definiert die Größe der Abbildung
current_dir = os.getcwd()

mol = Chem.MolFromSmiles(smiles) #Molecule from SMILES

molH=mol

if not Hda(mol): 
    molH=Chem.AddHs(mol)

molID=AllChem.EmbedMolecule(molH,randomSeed=435542)  # generate 3D coordinats 

# look for success

if (molID<0):
   sys.exit('Error in 3D coordinate generation. Program will exit.')

## optimize molecule
molOpt= ffoptimize(molH)

molCoded=DCodeName(molOpt)
## build distances
#molcoded is a molecule object with codes as an atomic property.
#Function call to store the code as an atomic property.
#As a second parameter, sort= Integer can be passed for the OZ of the atom type of interest.
#The default is 6 for carbon.
mol_mit_Code=DCodeMol(molCoded)
## DCode is now an atomic propertie
    

## build molecule
show_atom_number(molOpt, 'atomNote') #create numbering
mol_ohne_H = Chem.RemoveHs(molOpt)
AllChem.Compute2DCoords(mol_ohne_H)
img = Draw.MolToImage(mol_ohne_H, size=groesse)  # change size
## using tempfile library to create a temporary image to avoid problems updating the display
## molbild is a temporary name for the picture


with tempfile.NamedTemporaryFile(dir=os.getcwd(), suffix='.png', delete=False) as temp_file:
   molbild=temp_file.name
   img.save(molbild)  # save image



In [6]:
# read DCodes from properties

for atom in mol_mit_Code.GetAtoms():
    if atom.HasProp('DCode'):
        codestring=atom.GetProp('DCode')
        index=atom.GetIdx()+1
        verschiebung, trefferzahl, kuerzungen=calcshift(df,codestring)

## output as table

# initialise list
table_data = []

# collect data
for atom in mol_mit_Code.GetAtoms():
    if atom.HasProp('DCode'):
        codestring = atom.GetProp('DCode')
        index = atom.GetIdx() + 1
        verschiebung, trefferzahl, kuerzungen = calcshift(df, codestring)
        
        # insert data into table
        
        table_data.append({
            "Atom": index,
            "Shift [ppm]": f"{verschiebung:.2f}",
            "Hits": trefferzahl,
            "Shorts": kuerzungen - 1 if (kuerzungen - 1) != 0 else 0
        })

# create Pandas DataFrame
df_table = pd.DataFrame(table_data)

# show image and table
#display(Image(filename=molbild)) #show image

#display(HTML(df_table.to_html(escape=False, index=False)))

molbild_filename = os.path.basename(molbild)

# HTML-Template 
html_content = f"""
<div style="display: flex; align-items: flex-start;">
    <!-- Bild -->
    <div style="margin-right: 20px;">
        <img src="{molbild_filename}" alt="Molbild" style="width:450px;height:auto;">
    </div>
    
    <!-- Tabelle -->
    <div>
        {df_table.to_html(escape=False, index=False)}
    </div>
</div>
"""

# show HTML
display(HTML(html_content))

Atom,Shift [ppm],Hits,Shorts
2,54.00,3,13
3,28.46,1,11
4,34.40,5,18
5,176.40,1,16
8,58.40,1,20
9,-999.00,0,24
11,172.90,2,19
14,41.25,14,9
15,171.30,5,5
18,175.93,2,8


## Please wait for finishing output

In [16]:
## cleanup procedure
# start after output is visible
## remove temporary picture

os.remove(molbild)